# «Основы парсинга и работы с API»

# Задание 1

## Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

## Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [77]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from datetime import date, timedelta

In [63]:
url = 'https://habr.com/ru/all/'
keywords = ['python', 'парсинг']

In [64]:
def get_all_links(url):
    all_links = []
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    posts = soup.find_all('article', class_='post')
    for post in posts:
        post_link = post.find('a', class_='post__title_link').attrs.get('href')
        all_links.append(post_link)
    return all_links

In [83]:
def fix_date(broken_date):
    fixed_date = broken_date.replace('сегодня в', str(date.today())).replace('вчера в', str(date.today() - timedelta(days=1)))
    return fixed_date

In [86]:
def get_keyword_articles(links, keywords):
    articles = pd.DataFrame()
    for link in links:
        req = requests.get(link)
        soup = BeautifulSoup(req.text, 'html.parser')
        post = soup.find('article', class_='post')
        time.sleep(0.5)
        if any([kw.lower() in post.text.lower() for kw in keywords]):
            post_time = post.find('span', class_='post__time').text
            post_title = post.find('h1', class_='post__title').text
            post_body = post.find('div', class_='post__body').text
            row = {'date': fix_date(post_time), 'title': post_title, 'link': link, 'text': post_body}
            articles = pd.concat([articles, pd.DataFrame([row])])
    return articles

links = get_all_links(url)
get_keyword_articles(links, keywords)

,date,title,link,text
0,2020-10-11 17:10,\nFOSS News №37 – дайджест новостей и других м...,https://habr.com/ru/post/522958/,\n\n\r\nВсем привет!\n\r\nПродолжаем дайджесты...
0,2020-10-11 14:29,\n«Если уже слили»: как сделать документооборо...,https://habr.com/ru/company/1cloud/blog/522914/,"\nРанее мы обсуждали неочевидные моменты, связ..."
0,2020-10-11 13:58,\nНетривиальное слияние репозиториев с помощью...,https://habr.com/ru/post/522942/,\nЗадача\r\nДано: проект на основе OpenWRT (а ...
0,2020-10-11 09:34,\nКонструкторы Fischertechnik плюс одноплатный...,https://habr.com/ru/post/522922/,\nОтдавая дань продолжительному карантину этог...
0,2020-10-10 23:05,\n«Велосипедный набор» или о создании самодель...,https://habr.com/ru/post/522898/,\nНесколько лет назад я ненадолго увлекся маши...


# Задание 2

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. 

Список email-ов задаем переменной в начале кода:

EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [170]:
import json
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
params = {
    "emailAddresses":['ivankotov@gmail.com', 'ivankotov1989@gmail.com'],
}
headers = {
    'Vaar-Version':'0',
    'Vaar-Header-App-Product':'hackcheck-web-avast'
}
res = requests.post(url, headers=headers, data=json.dumps(params))
res.json()

breach_email = {}
df = pd.DataFrame()

for email, breaches in res.json()['summary'].items():
    breach_email[email] = list(breaches.values())[0]

for id_, details in res.json()['breaches'].items():
    for email, breaches in breach_email.items():
        if int(id_) in breaches:
            row = {'email': email, 
                   'date': details['publishDate'], 
                   'source': details['site'],
                   'description': details['description']
                  }
            df = pd.concat([df, pd.DataFrame([row])])

df

,email,date,source,description
0,ivankotov@gmail.com,2019-03-07T00:00:00Z,dubsmash.com,"In December 2018, Dubsmash's database was alle..."
0,ivankotov@gmail.com,2020-07-23T00:00:00Z,wattpad.com,"In June 2020, the online writing community Wat..."
